In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

1- importer les librairies

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import xgboost as xgb
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')

%matplotlib inline

2 - importer le dataset

In [ ]:
train = pd.read_csv ('../input/train.csv')
test = pd.read_csv ('../input/test.csv')
sample_submission = pd.read_csv ('../input/sample_submission.csv')

3- Etude du dataset

In [ ]:
train.head(20)

In [ ]:
train.tail(20)

In [ ]:
test.head(20)

In [ ]:
test.tail(20)

In [ ]:
sample_submission.head(20)

In [ ]:
sample_submission.tail(20)

In [ ]:
train.shape , test.shape, sample_submission.shape

In [ ]:
train.index , test.index , sample_submission.index 

In [ ]:
train.columns , test.columns , sample_submission.columns 

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
sample_submission.info()

Check ou and Detecting the missing Values

In [ ]:
train.isnull().sum()

In [ ]:
train.dropna(inplace=True)
train.isnull().sum()

In [ ]:
train.shape

In [ ]:
train.trip_duration.min()

In [ ]:
train.trip_duration.max()

see the categorical values

In [ ]:
import pandas as pd
train = pd.read_csv('../input/train.csv')
train

cleaning the data

In [ ]:
#gestion des outliers
plt.subplots(figsize=(17,7))
plt.title("outliers repartition")
train.boxplot()

In [ ]:
train.loc[train.trip_duration<5000,"trip_duration"].hist(bins=150
                                                        )

In [ ]:
# comme on remarque que les outliers sont rencontrés au niveau du trip_duration du coup nous allons nettoyer notre data suivant le trip_duration
m = np.mean(train['trip_duration'])
s = np.std(train['trip_duration'])
train = train[train['trip_duration'] <= m + 2*s]
train = train[train['trip_duration'] >= m - 2*s]

In [ ]:
#on remarque aussi la même chose au niveau de la longitude et latitude , nous allons donc nettoyer à nouveau 

train = train[train['pickup_longitude'] <= -73.75]
train = train[train['pickup_longitude'] <= -74.03]
train = train[train['pickup_latitude'] <= 40.85]
train = train[train['pickup_latitude'] <= 40.63]
train = train[train['dropoff_longitude'] <=  -73.75]
train = train[train['dropoff_longitude'] <= -74.03]
train = train[train['dropoff_latitude'] <= 40.85]
train = train[train['dropoff_latitude'] <= 40.63]

In [ ]:
# on regarde the categorical data
X = train.iloc[:,:-1].values
X

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X[:,0] = label_encoder.fit_transform(X[:,0])
X

Dat visualisation et analyse

In [ ]:
train['log_trip_duration'] = np.log(train['trip_duration'].values + 1)
plt.hist(train['log_trip_duration'].values, bins=50)
plt.xlabel('log(trip_duration)')
plt.ylabel('number of train records')
plt.show()
sns.distplot(train["log_trip_duration"], bins =200)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
plot_vendor = train.groupby('vendor_id')['trip_duration'].mean()
plt.subplots(1,1,figsize=(17,10))
plt.ylim(ymin=800)
plt.ylim(ymax=840)
sns.barplot(plot_vendor.index,plot_vendor.values)
plt.title('Time per Vendor')
plt.legend(loc=0)
plt.ylabel('Time in Seconds')

In [ ]:
snwflag = train.groupby('store_and_fwd_flag')['trip_duration'].mean()
plt.subplots(1,1,figsize=(18,10))
plt.ylim(ymin=0)
plt.ylim(ymax=1100)
plt.title('Time per store_and_fwd_flag')
plt.legend(loc=0)
plt.ylabel('Time in Seconds')
sns.barplot(snwflag.index,snwflag.values)

In [ ]:
train['log_trip_duration'] = np.log(train['trip_duration'].values + 1)
plt.hist(train['log_trip_duration'].values, bins=100)
plt.xlabel('log(trip_duration)')
plt.ylabel('number of train records')
plt.show()

In [ ]:
pc = train.groupby('passenger_count')['trip_duration'].mean()

plt.subplots(1,1,figsize=(17,10))
plt.ylim(ymin=0)
plt.ylim(ymax=1100)
plt.title('Time per store_and_fwd_flag')
plt.legend(loc=0)
plt.ylabel('Time in Seconds')
sns.barplot(pc.index,pc.values)


In [ ]:
import matplotlib.pyplot as plt 
from matplotlib import animation
from matplotlib import cm
import base64
import io
longitude = list(train.pickup_longitude) + list(train.dropoff_longitude)
latitude = list(train.pickup_latitude) + list(train.dropoff_latitude)
plt.figure(figsize = (10, 10))
plt.plot(longitude,latitude,'.', alpha = 1, markersize = 25)
plt.show()

In [ ]:
train.groupby('passenger_count').size()

In [ ]:
test.groupby('passenger_count').size()

In [ ]:
train.loc[train.trip_duration<5000,"trip_duration"].hist(bins=50
                                                              )

selection exctraction création 

In [ ]:
col_diff = list(set(train.columns).difference(set(test.columns)))
col_diff

In [ ]:
train['dist'] = abs((train['pickup_latitude']-train['dropoff_latitude'])
                        + (train['pickup_longitude']-train['dropoff_longitude']))
test['dist'] = abs((test['pickup_latitude']-test['dropoff_latitude'])
                        + (test['pickup_longitude']-test['dropoff_longitude']))

In [ ]:
y = train["trip_duration"]
train.drop(["trip_duration"], axis=1, inplace=True)
X = train
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, random_state=42)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

In [ ]:
randf = RandomForestRegressor()
randf.fit(X, y)